In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import os
import json
import time
from itertools import chain


#!conda config --env --add channels conda-forge
#!conda install numpy -y
#!conda install pandas -y
#!conda install matplotlib -y

-  Spotify Million Playlist Dataset Challenge https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#dataset
- How to Develop an End-to-End Machine Learning Project and Deploy it to Heroku with Flask https://www.freecodecamp.org/news/end-to-end-machine-learning-project-turorial/
- A Full End-to-End Deployment of a Machine Learning Algorithm into a Live Production Environment https://towardsdatascience.com/a-full-end-to-end-deployment-of-a-machine-learning-algorithm-into-a-live-production-environment-3d9971ade188
- Object Oriented Machine Learning — Create Production grade Machine Learning Pipeline using OOPs https://medium.com/analytics-vidhya/object-oriented-machine-learning-create-production-grade-machine-learning-pipeline-using-oops-50f3cfa14659

In [ ]:
path = "/home/maksim/Documents/ML/Datasets/Spotify_data/jsons/"
filenames = os.listdir(path)


N = 10
for j in range(N):
    playlist_track = pd.DataFrame()
    playlist = pd.DataFrame()
    track = pd.DataFrame()
    for filename in enumerate(filenames[1000//N * j:1000//N * (j+1)]):
        fullpath = os.sep.join((path, filename[1]))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        playlist_temp = pd.DataFrame(mpd_slice['playlists'])
        playlist_temp['tracks'] = pd.Series([[playlist_temp.tracks[j][i] | {'pid': playlist_temp.pid[j]} 
                                         for i in range(len(playlist_temp.tracks[j]))] 
                                        for j in range(len(playlist_temp))])
        track_temp = pd.DataFrame(list(chain.from_iterable(playlist_temp['tracks'])))
        playlist_track = pd.concat([playlist_track, track_temp[['pid','track_uri']]])
        track = pd.concat([track, track_temp]).drop_duplicates('track_uri')
        playlist = pd.concat([playlist, playlist_temp.drop(columns=['tracks'], axis = 1)])
    playlist_track.to_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_track_{}.csv'.format(j), encoding='utf-8', index=False)
    playlist.to_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_{}.csv'.format(j), encoding='utf-8', index=False)
    track.to_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/track_{}.csv'.format(j), encoding='utf-8', index=False)

In [6]:
df_playlist = pd.concat([pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_{}.csv'.format(i)) for i in range(10)])
df_track = pd.concat([pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/track_{}.csv'.format(i)) for i in range(10)])
df_playlist_track = pd.concat([pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_track_{}.csv'.format(i)) for i in range(10)])

In [24]:
df_track_unique = df_track.drop_duplicates('track_uri').drop('pid', axis=1)

df_playlist.to_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/track_full.csv', encoding='utf-8', index=False)
df_track_unique.to_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_full.csv', encoding='utf-8', index=False)
df_playlist_track.to_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_track_full.csv', encoding='utf-8', index=False)

In [ ]:
playlist_track_full

In [3]:
'track_full.csv'[:-9]

'track'

In [6]:
import psycopg2
import yaml
from sqlalchemy import create_engine
import pandas as pd


credentials = yaml.safe_load(open('/home/maksim/Documents/credentials.yml'))
pwd = credentials['maksim']['password']
conn = psycopg2.connect(database="spotify", user="maksim",
    password=pwd, host="localhost")
engine = create_engine('postgresql+psycopg2://maksim:{}@localhost/spotify'.format(pwd))

In [4]:
# playlist_full = pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_full.csv')
# track_full = pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/track_full.csv')
playlist_track_full = pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_track_full.csv')

In [ ]:
playlist_track_full.to_sql('playlist_track', con=engine, if_exists='replace', index=False)

In [99]:
playlist_track_full.to_sql('playlist_track', con=engine, if_exists='replace', index=False)

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Kaya Stewart,spotify:track:1u2tUlfNU6ncpaooRo7GN6,spotify:artist:0ay2fTD85rNUhJz8rlxE7a,In Love With a Boy,spotify:album:7qGZ5CPGP7KPDRMa98PcEj,173080,Kaya Stewart
1,1,Kelly Clarkson,spotify:track:0yUlMnLpU2W6JQtvg1k4Od,spotify:artist:3BmGtnKgCSGYIUhmivXKWX,Invincible,spotify:album:7oKtXc3FkeOZTCB88YugON,238360,Piece By Piece (Deluxe Version)
2,2,Ed Sheeran,spotify:track:0A2J5TumCpT4aJVvQHNEQW,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,One,spotify:album:6NoBzYmh5gUusGPCfg0pct,252760,x
3,3,Ed Sheeran,spotify:track:3KiktyOxQd2Z0bmnabgRq2,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I'm A Mess,spotify:album:6NoBzYmh5gUusGPCfg0pct,244573,x
4,4,Ed Sheeran,spotify:track:6u0oHknFYrnUd42A6rvFly,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,Sing,spotify:album:6NoBzYmh5gUusGPCfg0pct,235382,x
...,...,...,...,...,...,...,...,...
2262287,74,Randy Crawford,spotify:track:2HB63alJ0a0H8OIx8ObTai,spotify:artist:1twC2fwPG5FkvYcMpVBQRz,I'm Glad There Is You,spotify:album:5xTMlKzOVtWESRouQ0pZZy,129306,Don't Say It's Over
2262288,43,Waa Wei,spotify:track:2nm6SuuxlBT2q1uygJy31L,spotify:artist:190bkHbFrRvEhcB7Zpuv3y,Can't You Be a Tree,spotify:album:6Pchl8yElE3UoWPhSAeNw5,319013,Graceful Porcupine
2262289,44,Nathan Riki Thomson,spotify:track:3pAK7sF6u0ubzTcNHVt16b,spotify:artist:3JzPrHHcHHsNyBNE6poX0I,Hello Porcupine,spotify:album:6glaRMKQttFMDhLwpLTYLG,182271,Under Ubi's Tree
2262290,45,Why They Came,spotify:track:7Auq4yduwQgM1PnXuHkmWi,spotify:artist:4HxsA0CBRqEqxlRjqx3fGi,I Give Up (said the porcupine),spotify:album:7CxrgNkpyVQZgYnrrKPx7z,223973,Don't Eat From the Candy Tree


In [ ]:
df = pd.read_csv('/home/maksim/Documents/ML/Datasets/Spotify_data/playlist_track_full.csv')
engine = create_engine('postgresql+psycopg2://maksim:{}@localhost/spotify'.format(pwd))
df.to_sql('playlist', con=engine, if_exists='replace', index=False)

In [67]:
tuples = [tuple(x) for x in df.to_numpy()]
cols = ','.join(list(df.columns))
query = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
cursor = conn.cursor()
cursor.executemany(query, tuples)
conn.commit()
cursor.close()

TypeError: not all arguments converted during string formatting